In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 2, 17)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-02-17
yesterday: 2025-02-14


In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Tables in the process

In [10]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [19]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,MCS,43,2023-05-17,"3,000",7.60,1079,1253,2024-09-25,8.05,49.99,3.50,"24,096.51",497,"1,246.01",5.45%,4.00%,-79,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [5]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'MCS'

In [6]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1253



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,MCS,1253,43,2023-05-17,"3,000",7.60,47.20,3.30,Sold,"22,850.50",DTD,None


In [7]:
transaction, name, qty, sell_price, buy_price

('S', 'MCS', -3000, 8.05, 7.6)

### In case of buy, retreive buy id from latest buy record (id)

In [12]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,ORI,1315,125,2025-02-17,"10,000",2.90,60.03,4.20,Active,"29,064.23",DTD,None


In [14]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'ORI', 10000, 2.9)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [8]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'MCS'



,name,date,shares,price,active,period,grade
0,MCS,2016-09-20,"78,000",15.10,1,2,A4


In [9]:
#qty = -4800
new_cost = 15.40
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -3000, price = 15.4
WHERE name = 'MCS'



In [10]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [11]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'MCS'



,name,date,shares,price,active,period,grade
0,MCS,2016-09-20,"75,000",15.40,1,2,A4


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [12]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'MCS'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MCS,0.3200,0.0000,0.2000,0.0000,0.5200,"78,000",2024-08-23,2024-09-10,,1


In [13]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -3000
WHERE name = 'MCS'



In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [15]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'MCS'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MCS,0.3200,0.0000,0.2000,0.0000,0.5200,"75,000",2024-08-23,2024-09-10,,1


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [16]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'MCS'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,114,MCS,0.00,0.00,B,6.00,8.35,0.00,0.00,15.10,"4,000",-4,4,"78,000",0.000000,0.000000,RND60pct,SET


In [17]:
buy_target  = 7.40 # must calculate manually beforehand
sell_target = 12.90
cost = 15.40
status = 'B'
reason = "RD50Pct"
sell_qty = 5000

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, cost = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, cost, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -3000,
qty = 5000, buy_target = 7.4, sell_target = 12.9, cost = 15.4, reason = 'RD50Pct', status = 'B'
WHERE name = 'MCS'



In [18]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [19]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'MCS'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,114,MCS,0.00,0.00,B,7.40,12.90,0.00,0.00,15.40,"5,000",-4,4,"75,000",0.000000,0.000000,RD50Pct,SET


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [26]:
sql = """
SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = 'ORI'



,name,date,shares,price,active,dividend,period,grade,cost_amt
0,ORI,2022-04-07,50000,10.4,1,0.321,1,C3,520000.0


In [28]:
buy.dtypes

name                object
date        datetime64[ns]
shares               int64
price              float64
active               int64
dividend           float64
period              object
grade               object
cost_amt           float64
dtype: object

In [22]:
print(transaction, name, qty, buy_price)

B ORI 10000 2.9


In [42]:
new_unit_cost = buy['cost_amt'] + (qty * buy_price)
new_qty = buy['shares'] + qty
print(new_unit_cost)
print(new_qty)
new_unit_cost = new_unit_cost / new_qty
print(new_unit_cost)

0    549000.0
Name: cost_amt, dtype: float64
0    60000
Name: shares, dtype: int64
0    9.15
dtype: float64


### In case there is the same stock name in port

In [48]:
def update_buy(const, name, new_qty, new_unit_cost):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty.iloc[0]
    new_unit_cost_value = new_unit_cost.iloc[0]
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty, 
            price = :new_unit_cost
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

# Example usage
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [50]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'ORI'



,name,date,shares,price,active,dividend,period,grade
0,ORI,2022-04-07,"60,000",9.15,1,0.3210,1,C3


### In case of totally new stock

In [11]:
rcd = ['TVO', '2024-11-06', 4000, 24.90, 0, 0, 1, 0, 4,'A2']
rcd

['TVO', '2024-11-06', 4000, 24.9, 0, 0, 1, 0, 4, 'A2']

In [12]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [13]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [15]:
name = 'TVO'

In [16]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'TVO'



,name,date,shares,price,active,dividend,period,grade
0,TVO,2024-11-06,"4,000",24.90,1,0.0000,4,A2


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [52]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'ORI' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.3000,0.0210,0.0000,0.0000,0.3210,"50,000",2024-11-28,2024-12-13,,0


In [54]:
dividend.dtypes

name                object
q4                 float64
q3                 float64
q2                 float64
q1                 float64
dividend           float64
shares               int64
xdate       datetime64[ns]
paiddate    datetime64[ns]
kind                object
actual               int64
dtype: object

In [56]:
new_qty = dividend['shares'] + qty
print(new_qty)

0    60000
Name: shares, dtype: int64


In [62]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty.iloc[0]    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [64]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [66]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'ORI' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ORI,0.3000,0.0210,0.0000,0.0000,0.3210,"60,000",2024-11-28,2024-12-13,,0


### In case of totally new stock

In [15]:
rcd = ['TVO', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]
rcd

['TOA', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]

In [15]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [16]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [17]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'TOA' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TOA,0.2500,0.0000,0.3300,0.0000,0.5800,"1,000",2023-08-30,2023-09-14,,1


### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [106]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'ORI'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,116,ORI,0.0,0.0,X,2.9,11.1,0.0,0.0,10.4,10000,-4,4,50000,0.0,0.0,RD70Pct,SET100


In [108]:
stock.dtypes

id                 int64
name              object
max_price        float64
min_price        float64
status            object
buy_target       float64
sell_target      float64
volume           float64
beta             float64
cost             float64
qty                int64
buy_spread         int64
sell_spread        int64
available_qty      int64
bl               float64
sh               float64
reason            object
market            object
dtype: object

In [134]:
#
new_buy_target = 2.50
new_sell_target = 9.15
new_buy_qty = 10_000
print(transaction, name, qty, buy_price, new_buy_target, new_sell_target, new_buy_qty) 

B ORI 10000 2.9 2.5 9.15 10000


In [142]:
print(new_qty.iloc[0])

60000


In [150]:
new_qty.dtypes

dtype('int64')

In [122]:
print(new_unit_cost.iloc[0])

9.15


In [152]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    new_unit_cost_value = float(new_unit_cost.iloc[0])   
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    rp = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"
# Example usage
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [154]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'ORI'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,116,ORI,0.0,0.0,X,2.5,9.15,0.0,0.0,9.15,10000,-4,4,60000,0.0,0.0,RD70Pct,SET100


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

### =================================================

In [156]:
price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (price_date)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-02-14' ORDER BY period, buy.name


In [158]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,14.00,"44,400.00","16,800.00","-27,600.00",-62.16
1,1,C1,ASK,2023-02-16,"6,000",26.25,7.60,"157,500.00","45,600.00","-111,900.00",-71.05
2,1,C1,ASP,2022-03-07,"30,000",3.80,2.30,"114,000.00","69,000.00","-45,000.00",-39.47
3,1,C3,IVL,2022-08-29,"7,200",40.00,23.00,"288,000.00","165,600.00","-122,400.00",-42.50
4,1,C3,ORI,2022-04-07,"60,000",9.15,2.90,"549,000.00","174,000.00","-375,000.00",-68.31
5,1,C4,PTTGC,2021-03-17,"6,000",64.75,19.40,"388,500.00","116,400.00","-272,100.00",-70.04
6,1,C2,SCC,2021-09-20,600,405.00,145.50,"243,000.00","87,300.00","-155,700.00",-64.07
7,1,A2,SINGER,2023-01-19,"6,000",24.80,5.80,"148,800.00","34,800.00","-114,000.00",-76.61
8,1,C2,STA,2021-06-15,"10,000",30.00,15.90,"300,000.00","159,000.00","-141,000.00",-47.00
9,1,C1,TMT,2021-08-16,"36,000",10.20,3.04,"367,200.00","109,440.00","-257,760.00",-70.20


In [160]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [162]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [164]:
output.to_csv(output_file, header=True, index=False)
output.to_csv(icd_file, header=True, index=False)
output.to_csv(god_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)

In [166]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

output.to_excel(xsl_file, index=False)